In [1371]:
import pandas as pd

# EXTRACTION

### 1. Extraction of data from excel to dataframe using panda
### 2. create a workcopy of the extracted data

In [1372]:
#extracting call log data
call_log= pd.read_csv(r"C:\Users\JOY\Documents\Data Engineering python\ETL PROJECTS\Final Project\Final Project\call log.csv")
call_log.head()

,id,callerID,agentID,complaintTopic,assignedTo,status,resolutionDurationInHours
0,1,CALLER_1,163,Billing discrepancy,122.0,CLOSED,33.0
1,2,CALLER_2,133,Difficulty reaching customer support,NaN,new,NaN
2,3,CALLER_3,153,Wrong item received,122.0,resolved,NaN
3,4,CALLER_4,161,Unauthorized charges,151.0,pEnding,NaN
4,5,CALLER_5,148,Unprofessional behavior,NaN,new,NaN


In [1373]:
# creating a copy of call log data 

logcopy=call_log.copy()

In [1374]:
# creating a copy of call details data
call_details= pd.read_csv(r"C:\Users\JOY\Documents\Data Engineering python\ETL PROJECTS\Final Project\Final Project\call details.csv")


In [1375]:
# creating a copy of call details data 
detailcopy= call_details.copy()

In [1376]:
detailcopy.head()

,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent
0,ageentsGradeLevel,97,A,Inbound,False
1,2,64,E,in-bound,True
2,3,100,C,in-bound,False
3,4,69,D,Outbound,True
4,5,96,D,Inbound,True


# Merging both data frames

##### Change the datatype on detailcopy from strings to int

In [1377]:
#cleaning the callID column with a wrong value of "ageentGradelevel"
detailcopy['callID'][0]=1
detailcopy.head()

C:\Users\JOY\AppData\Local\Temp\ipykernel_2768\1135166459.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  detailcopy['callID'][0]=1


,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent
0,1,97,A,Inbound,False
1,2,64,E,in-bound,True
2,3,100,C,in-bound,False
3,4,69,D,Outbound,True
4,5,96,D,Inbound,True


In [1378]:
detailcopy.dtypes

callID                   object
callDurationInSeconds     int64
agentsGradeLevel         object
callType                 object
callEndedByAgent           bool
dtype: object

In [1379]:
#making both columns the same data types so as to merge
detailcopy['callID']=pd.to_numeric(detailcopy['callID']) 
detailcopy.dtypes

callID                    int64
callDurationInSeconds     int64
agentsGradeLevel         object
callType                 object
callEndedByAgent           bool
dtype: object

In [1380]:
logcopy.dtypes

id                             int64
callerID                      object
agentID                        int64
complaintTopic                object
assignedTo                   float64
status                        object
resolutionDurationInHours    float64
dtype: object

##### Merging both data frames on 'callID' for detailcopy and 'id' on logcopy

In [1381]:
# merging the data frames
merged_df=pd.merge(detailcopy,logcopy, left_on='callID', right_on='id')
merged_df.head(5)

,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent,id,callerID,agentID,complaintTopic,assignedTo,status,resolutionDurationInHours
0,1,97,A,Inbound,False,1,CALLER_1,163,Billing discrepancy,122.0,CLOSED,33.0
1,2,64,E,in-bound,True,2,CALLER_2,133,Difficulty reaching customer support,NaN,new,NaN
2,3,100,C,in-bound,False,3,CALLER_3,153,Wrong item received,122.0,resolved,NaN
3,4,69,D,Outbound,True,4,CALLER_4,161,Unauthorized charges,151.0,pEnding,NaN
4,5,96,D,Inbound,True,5,CALLER_5,148,Unprofessional behavior,NaN,new,NaN


# DATA CLEANING

### 1. Clean the inconsistencies in string columns


In [1382]:
merged_df.head(2)

,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent,id,callerID,agentID,complaintTopic,assignedTo,status,resolutionDurationInHours
0,1,97,A,Inbound,False,1,CALLER_1,163,Billing discrepancy,122.0,CLOSED,33.0
1,2,64,E,in-bound,True,2,CALLER_2,133,Difficulty reaching customer support,NaN,new,NaN


In [1383]:
#consistency in callType column
merged_df['callType']=merged_df['callType'].str.replace('-', '')
merged_df['callType']=merged_df['callType'].str.strip('')

In [1384]:
#consistency in callType column
merged_df['callType']=merged_df['callType'].apply(str.lower)
merged_df['callType']

0       inbound
1       inbound
2       inbound
3      outbound
4       inbound
         ...   
595    outbound
596     inbound
597    outbound
598     inbound
599    outbound
Name: callType, Length: 600, dtype: object

In [1385]:
#consistency in status column
merged_df['status']=merged_df['status'].apply(str.upper)

### Changing time columns datatypes to timestamps

In [1386]:
# changing "callDurationInSeconds" to timestamp
merged_df['callDurationInSeconds']=merged_df['callDurationInSeconds'].apply(str)
merged_df['callDurationInSeconds']=pd.to_datetime(merged_df['callDurationInSeconds'], unit='s')
merged_df['callDurationInSeconds']=merged_df['callDurationInSeconds'].dt.strftime("%H:%M:%S")
merged_df['callDurationInSeconds']
#df['A'] = pd.to_datetime(df["A"], unit='s').dt.strftime("%H:%M:%S")

C:\Users\JOY\AppData\Local\Temp\ipykernel_2768\66569766.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  merged_df['callDurationInSeconds']=pd.to_datetime(merged_df['callDurationInSeconds'], unit='s')


0      00:01:37
1      00:01:04
2      00:01:40
3      00:01:09
4      00:01:36
         ...   
595    00:02:27
596    00:01:30
597    00:01:29
598    00:02:26
599    00:01:43
Name: callDurationInSeconds, Length: 600, dtype: object

### 3. Filling NaN values


In [ ]:
# filling Null values
# changing "callDurationInSeconds" to timestamp
merged_df['resolutionDurationInHours']=merged_df['resolutionDurationInHours'].fillna(0)
merged_df['resolutionDurationInHours']=merged_df['resolutionDurationInHours'].replace('NaN','00:00:00.000')
merged_df['resolutionDurationInHours']=pd.to_timedelta(merged_df['resolutionDurationInHours'], unit='h')
merged_df['resolutionDurationInHours'].head(15)
#merged_df['resolutionDurationInHours']


##### In a cases where a call was not assigned to as agent, it is assumed that the case was handled by the agent who took the call

In [ ]:
# filling null values in column assigned to with agent 
#merged_df['assignedTo']=merged_df['assignedTo'].fillna(0)
#merged_df['assignedTo']=merged_df['assignedTo'].astype(int)
merged_df['assignedTo']=merged_df['assignedTo'].fillna(merged_df['agentID'])
merged_df['assignedTo']=merged_df['assignedTo'].astype(int)
merged_df['assignedTo'].head(20)

In [1389]:
merged_df.head(1)

,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent,id,callerID,agentID,complaintTopic,assignedTo,status,resolutionDurationInHours
0,1,00:01:37,A,inbound,False,1,CALLER_1,163,Billing discrepancy,122,CLOSED,1 days 09:00:00


In [1390]:
merged_df.drop('id', inplace=True, axis=1)

In [1391]:
#my_merged_df=['callID','callerID', 'callDurationInSeconds', 'agentsGradeLevel', 'callType', 
              #'callEndedByAgent', 'agentID', 'complaintTopic', 'assignedTo', 'status', 'resolutionDurationInHours']
merged_df

,callID,callDurationInSeconds,agentsGradeLevel,callType,callEndedByAgent,callerID,agentID,complaintTopic,assignedTo,status,resolutionDurationInHours
0,1,00:01:37,A,inbound,False,CALLER_1,163,Billing discrepancy,122,CLOSED,1 days 09:00:00
1,2,00:01:04,E,inbound,True,CALLER_2,133,Difficulty reaching customer support,133,NEW,0 days 00:00:00
2,3,00:01:40,C,inbound,False,CALLER_3,153,Wrong item received,122,RESOLVED,0 days 00:00:00
3,4,00:01:09,D,outbound,True,CALLER_4,161,Unauthorized charges,151,PENDING,0 days 00:00:00
4,5,00:01:36,D,inbound,True,CALLER_5,148,Unprofessional behavior,148,NEW,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...
595,596,00:02:27,E,outbound,True,CALLER_596,137,Unhelpful support,169,NEW,0 days 00:00:00
596,597,00:01:30,A,inbound,True,CALLER_597,159,Slow response time,159,RESOLVED,0 days 00:00:00
597,598,00:01:29,F,outbound,True,CALLER_598,148,Shipping delay,150,PENDING,0 days 00:00:00
598,599,00:02:26,D,inbound,False,CALLER_599,161,Website not working,148,PENDING,0 days 00:00:00


In [1392]:
#export dataframe to csv

#merged_df.to_csv('merged_cleaned_data.csv', index=False)

## LOADING CLEANED CSV TO S3

In [1393]:
import boto3
s3 = boto3.client('s3')
s3.upload_file('merged_cleaned_data.csv', 'myweserve', 'weservedata.jpg')


NoCredentialsError: Unable to locate credentials